In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from amberdata_derivatives import AmberdataDerivatives
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
import mplfinance as mpf
import numpy as np
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
# insert API key below
amberdata_client = AmberdataDerivatives(api_key="")

amberdata_client.get

In [ ]:
# Plot BTC <> ETH correlation of log daily returns 
data = amberdata_client.get_realized_volatility_correlation_beta(exchange='gdax', pair='btc_usd', pair2='eth_usd')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.timestamp, unit='ms')
data = data[::-1]

plt.figure(figsize=(15, 7))
plt.plot(data.correlation180)
plt.title("BTC <> ETH 180 Day RV Rolling Correlation: " + str(data.index[0]) + " to " + str(data.iloc[-1].name))
plt.ylabel("Rolling Correlation")

In [ ]:
# Plot BTC Monthly vs Daily Realized Volatility 
data = amberdata_client.get_realized_volatility_monthly_vs_daily_ratio(exchange='gdax', pair='btc_usd')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.timestamp, unit='ms')

# only look at a subset of data
data = data[:1000][::-1]
plt.figure(figsize=(15, 7))
plt.title("BTC Monthly vs Daily RV: " + str(data.index[0]) + " to " + str(data.iloc[-1].name))
plt.plot(data.monthlyHistoricalVolatility, label='Monthly RV')
plt.plot(data.dailyHistoricalVolatility30Days, label='Daily RV')
plt.ylabel("Realized Volatility (%)")
plt.legend();

In [ ]:
# plot BTC vol seasonality for day of week
data = amberdata_client.get_realized_volatility_seasonality_day_of_week(exchange='gdax', pair='btc_usd', startDate='2024-01-01', endDate='2024-05-01')
data = pd.DataFrame(data['payload']['data'])
data = data[::-1]

plt.figure(figsize=(15, 7))
plt.bar(data.weekday, data.historicalVolatility1day)
plt.ylabel("Average Realized Volatility (%)")
plt.title("BTC Seasonality: Volatility Day of Week: 2024-01-01 to 2024-05-01");

In [ ]:
# plot BTC vol seasonality for month of year
data = amberdata_client.get_realized_volatility_seasonality_month_of_year(exchange='gdax', pair='btc_usd', startDate='2024-01-01', endDate='2024-05-01')
data = pd.DataFrame(data['payload']['data'])

plt.figure(figsize=(15, 7))
plt.bar(data.month, data.historicalVolatility1day)
plt.ylabel("Average Realized Volatility (%)")
plt.title("BTC Seasonality: Volatility Month of Year: 2024-01-01 to 2024-05-01");

In [ ]:
# plot realized volatility cones for BTC 
data = amberdata_client.get_realized_volatility_cones(exchange='gdax', pair='btc_usd', startDate='2024-01-01', endDate='2024-05-01')
data = pd.DataFrame(data['payload']['data'])
data = data.iloc[:, 2:].T

data['dte'] = [
    int(''.join([char for char in column.split("_")[1] if char.isdigit()]))
    for column in data.index
]
data['vol_level'] = [data.index[i].split("_")[0] for i in range(len(data))]
data.columns = ['rv', 'dte', 'vol_level']

cone_max = data[data.vol_level=='max'].sort_values('dte', ascending=True)
cone_min = data[data.vol_level=='min'].sort_values('dte', ascending=True)
cone_25p = data[data.vol_level=='p25'].sort_values('dte', ascending=True)
cone_50p = data[data.vol_level=='p50'].sort_values('dte', ascending=True)
cone_75p = data[data.vol_level=='p75'].sort_values('dte', ascending=True)
cone_current = data[data.vol_level=='current'].sort_values('dte', ascending=True)

plt.figure(figsize=(15, 7))
plt.plot(cone_max.dte, cone_max.rv, label='Maximum')
plt.plot(cone_25p.dte, cone_25p.rv, label='25th Percentile')
plt.plot(cone_50p.dte, cone_50p.rv, label='50th Percentile')
plt.plot(cone_75p.dte, cone_75p.rv, label='75th Percentile')
plt.plot(cone_current.dte, cone_current.rv, label='Current')
plt.plot(cone_min.dte, cone_min.rv, label='Minimum')

plt.legend()
plt.title("BTC RV Volatility Cone: 2024-01-01 to 2024-05-01")
plt.ylabel("Realized Volatility (%)")
plt.xlabel("DTE")

In [ ]:
# plot IV vs RV + VRP 
data = amberdata_client.get_realized_volatility_implied_vs_realized(exchange='deribit', currency='btc', startDate='2024-01-01', endDate='2024-01-07')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.timestamp, unit='ms')
data['vrp'] = data.atm7 - data.realizedVolatility7

plt.figure(figsize=(15, 7))
plt.title("BTC ATM IV vs RV [7 DTE]: 2024-01-01 to 2024-01-05")
plt.bar(data.index, data.vrp, label='VRP [ATM - RV]', edgecolor='black', width=0.03)
plt.plot(data.index, data.atm7, label='ATM 7 DTE')
plt.plot(data.index, data.realizedVolatility7, label='RV 7 DTE')

plt.ylabel("Volatility (%)")
plt.legend()
plt.tight_layout()